In [53]:
import h5py as h5
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import sys
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from auton_survival.preprocessing import Scaler
import optuna
from sklearn.model_selection import ParameterGrid
#sys.path.append('/projects/EKOLEMEN/survival_tm/train_models/auton-survival')
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from sklearn.model_selection import ParameterGrid
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from auton_survival.models.dsm.dsm_torch import DeepSurvivalMachinesTorch
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [54]:
with open('models/simple_models.pkl', 'rb') as file:
    loaded_model = pickle.load(file)[1][0]

In [55]:
with open('data/x_train_df_normed.pkl', 'rb') as f:
    x_train_df = pickle.load(f)

In [60]:
a = loaded_model._model._gen_torch_model(179, 'Adam', 1)
#torch.save(a, 'temp.pth')
print(a.embedding)
#print(a.act)
print(a.shapeg['1'])
#print(a.shape['1'].expand(179, -1))
print(a.scaleg['1'])
#print(a.scale['1'].expand(179, -1))
print(a.gate['1'])
print(a.temp)

Sequential(
  (0): Linear(in_features=179, out_features=1000, bias=False)
  (1): ReLU6()
  (2): Linear(in_features=1000, out_features=1000, bias=False)
  (3): ReLU6()
)
Sequential(
  (0): Linear(in_features=1000, out_features=3, bias=True)
)
Sequential(
  (0): Linear(in_features=1000, out_features=3, bias=True)
)
Sequential(
  (0): Linear(in_features=1000, out_features=3, bias=False)
)
1.0


In [82]:
import torch
import torch.nn as nn

# Assuming 'a' is your original model
class EmbeddingModel(nn.Module):
    def __init__(self, original_model):
        super(EmbeddingModel, self).__init__()
        self.embedding = original_model.embedding
        # onnx2keras doesn't support relu6
        self.embedding[1] = nn.ReLU()
        self.embedding[3] = nn.ReLU()
    def forward(self, x):
        return self.embedding(x)

class ShapegModel(nn.Module):
    def __init__(self, original_model):
        super(ShapegModel, self).__init__()
        self.shapeg = original_model.shapeg['1']  # Access the specific module

    def forward(self, x):
        return self.shapeg(x)
    
class ScalegModel(nn.Module):
    def __init__(self, original_model):
        super(ScalegModel, self).__init__()
        self.scaleg = original_model.scaleg['1']  # Access the specific module

    def forward(self, x):
        return self.scaleg(x)
    
class GateModel(nn.Module):
    def __init__(self, original_model):
        super(GateModel, self).__init__()
        self.gate = original_model.gate['1']  # Access the specific module

    def forward(self, x):
        return self.gate(x)

scaleg = ScalegModel(a)
gate = GateModel(a)
shapeg = ShapegModel(a)
embedding = EmbeddingModel(a)

In [63]:
import onnx

# make a dummy input with the correct dimensions for your model
dummy_input = torch.randn(1, 179)
torch.onnx.export(embedding, dummy_input, 'embedding.onnx', verbose=True, input_names = ['input'], output_names = ['output'])

def sanitize_name(name):
    return name.lstrip('/').replace('/', '_')

def rename_nodes(model):
    for node in model.graph.node:
        node.name = sanitize_name(node.name)
        node.input[:] = [sanitize_name(inp) for inp in node.input]
        node.output[:] = [sanitize_name(out) for out in node.output]
    for input in model.graph.input:
        input.name = sanitize_name(input.name)
    for output in model.graph.output:
        output.name = sanitize_name(output.name)
    for initializer in model.graph.initializer:
        initializer.name = sanitize_name(initializer.name)

onnx_model = onnx.load('embedding.onnx')
rename_nodes(onnx_model)
onnx.save(onnx_model, 'sanitized_embedding.onnx')

from onnx2keras import onnx_to_keras
import onnx
onnx_model = onnx.load('sanitized_embedding.onnx')

keras_model = onnx_to_keras(onnx_model, ['input'])
keras_model.save('embedding.h5')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight onnx::MatMul_9 with shape (179, 1000).
DEBUG:onnx2keras:Found weight onnx::MatMul_10 with shape (1000, 1000).
DEBUG:onnx2keras:Found input input with shape [179]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MatMul
DEBUG:onnx2keras:node_name: embedding_embedding.0_MatMul_output_0
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input).
DEBUG:onnx2keras:Check input 1 (name onnx::MatMul_9).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights

Exported graph: graph(%input : Float(1, 179, strides=[179, 1], requires_grad=0, device=cpu),
      %onnx::MatMul_9 : Float(179, 1000, strides=[1, 179], requires_grad=0, device=cpu),
      %onnx::MatMul_10 : Float(1000, 1000, strides=[1, 1000], requires_grad=0, device=cpu)):
  %/embedding/embedding.0/MatMul_output_0 : Float(1, 1000, strides=[1000, 1], requires_grad=1, device=cpu) = onnx::MatMul[onnx_name="/embedding/embedding.0/MatMul"](%input, %onnx::MatMul_9), scope: __main__.EmbeddingModel::/torch.nn.modules.container.Sequential::embedding/torch.nn.modules.linear.Linear::embedding.0 # /home/hf8585/.conda/envs/torch/lib/python3.11/site-packages/torch/nn/modules/linear.py:116:0
  %/embedding/embedding.1/Relu_output_0 : Float(1, 1000, strides=[1000, 1], requires_grad=1, device=cpu) = onnx::Relu[onnx_name="/embedding/embedding.1/Relu"](%/embedding/embedding.0/MatMul_output_0), scope: __main__.EmbeddingModel::/torch.nn.modules.container.Sequential::embedding/torch.nn.modules.activation.Re

/home/hf8585/.conda/envs/torch/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [64]:
# convert relu to relu6 in keras model
from keras.models import Sequential
from keras.layers import ReLU

# Assume 'model' is your pre-loaded Keras sequential model
for i, layer in enumerate(keras_model.layers):
    if isinstance(layer, ReLU):
        keras_model.layers[i] = ReLU(max_value=6, name=layer.name)  # Replacing ReLU with ReLU6

# Save the updated model if necessary
keras_model.save('embedding.h5')


/home/hf8585/.conda/envs/torch/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [67]:
x_train = np.array(x_train_df)
print(x_train.shape)

(410234, 179)


In [74]:
x_train_tensor = torch.tensor(x_train[0:1]).float()
print(x_train_tensor.shape)

torch.Size([1, 179])


In [77]:
print(a(x_train_tensor))

(tensor([[0.9874, 1.0554, 1.0971]], grad_fn=<AddBackward0>), tensor([[1.1052, 0.9778, 0.9912]], grad_fn=<AddBackward0>), tensor([[ 0.1165,  0.0377, -0.0991]], grad_fn=<DivBackward0>))


In [85]:
xrep = embedding.forward(x_train_tensor)

dim = x_train_tensor.shape[0]
print(a.act(shapeg(xrep))+a.shape['1'].expand(dim, -1),
           a.act(scaleg(xrep))+a.scale['1'].expand(dim, -1),
           gate(xrep)/a.temp)


tensor([[0.9874, 1.0554, 1.0971]], grad_fn=<AddBackward0>) tensor([[1.1052, 0.9778, 0.9912]], grad_fn=<AddBackward0>) tensor([[ 0.1165,  0.0377, -0.0991]], grad_fn=<DivBackward0>)


In [105]:
from keras.models import load_model
x_train_keras = np.array(x_train_tensor)
embedding_keras = load_model('embedding.h5')
shapeg_keras = load_model('shapeg.h5')
scaleg_keras = load_model('scaleg.h5')
gate_keras = load_model('gate.h5')
print(x_train_keras.shape)
xred_keras = embedding_keras.predict(x_train_keras)

print(np.tanh(shapeg_keras.predict(xred_keras))+np.array([1,1,1]), 
      np.tanh(scaleg_keras.predict(xred_keras))+np.array([1,1,1]),
        gate_keras.predict(xred_keras))

(1, 179)
1/1 [==============================] - 0s 24ms/step
[[0.98744142 1.05535991 1.097114  ]] [[1.10518538 0.97778107 0.99118527]] [[ 0.11653557  0.03767988 -0.09908683]]


In [ ]:
# keras 2 c!
from keras2c import k2c

k2c(keras_model, 'k2c_model', malloc=False, num_tests=10, verbose=True)

In [71]:
b = torch.load('temp.pth')
print(b.forward(x_train_tensor.float()))

(tensor([[0.9874, 1.0554, 1.0971]], grad_fn=<AddBackward0>), tensor([[1.1052, 0.9778, 0.9912]], grad_fn=<AddBackward0>), tensor([[ 0.1165,  0.0377, -0.0991]], grad_fn=<DivBackward0>))


In [ ]:
def _lognormal_cdf(model, x, t_horizon, risk='1'):

  squish = nn.LogSoftmax(dim=1)

  shape, scale, logits = model.forward(x, risk)
  logits = squish(logits)

  k_ = shape
  b_ = scale

  t_horz = torch.tensor(t_horizon).double().to(logits.device)
  t_horz = t_horz.repeat(shape.shape[0], 1)

  cdfs = []

  # for j in range(len(t_horizon)):
  for j in range(t_horz.shape[1]):

    t = t_horz[:, j]
    lcdfs = []

    for g in range(model.k):

      mu = k_[:, g]
      sigma = b_[:, g]

      s = torch.div(torch.log(t) - mu, torch.exp(sigma)*np.sqrt(2))
      s = 0.5 - 0.5*torch.erf(s)
      s = torch.log(s)
      lcdfs.append(s)

    lcdfs = torch.stack(lcdfs, dim=1)
    lcdfs = lcdfs+logits
    lcdfs = torch.logsumexp(lcdfs, dim=1)
    cdfs.append(lcdfs.detach().cpu().numpy())

  return cdfs